In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [ ]:
### Neurons, Layers, and Models

### neuron network Module: nn.Module, a module can be a single layer, a component consisting of multiple layers, or the entire model itself
# Individual layers can be modules. Many layers can comprise a module. Many modules can comprise a module.

In [2]:
# nn.Sequential defines a special kind of Module, It maintains an ordered list of constituent Modules.
## LazyLinear class is also a subclass of Module.

net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand(2, 20)
net(X).shape

c:\Users\huiya\pytorch-basics\.venv311\Lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 10])

In [4]:
# A custom Module

class MLP(nn.Module):
    def __init__(self):
        # Call the constructor of the parent class nn.Module to perform
        # the necessary initialization
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input X
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X).shape

torch.Size([2, 10])

In [7]:
# The Sequential Module
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X

In [8]:
net = MySequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net(X).shape

torch.Size([2, 10])

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20))
        self.linear = nn.LazyLinear(20)

    def forward(self, X):
        X = self.linear(X)

        # A hidden layer whose weights (self.rand_weight) are initialized randomly at instantiation and are thereafter constant. 
        X = F.relu(X @ self.rand_weight + 1)
        
        # Reuse the fully connected layer. This is equivalent to sharing parameters with two fully connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0664, grad_fn=<SumBackward0>)

In [10]:
# We can mix and match various ways of assembling modules together.

class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                 nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)

tensor(0.2779, grad_fn=<SumBackward0>)

========Parameter Management================

In [11]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

In [13]:
# When a model is defined via the Sequential class, we can first access any layer by indexing into the model as though it were a list.

net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.0980, -0.0327, -0.1445,  0.1823,  0.1344, -0.1969,  0.2621, -0.3496]])),
             ('bias', tensor([0.1680]))])

In [14]:
# each parameter is represented as an instance of the parameter class.

type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([0.1680]))

In [15]:
# Parameters are complex objects, containing values, gradients, and additional information. That is why we need to request the value explicitly.

net[2].weight.grad == None


True

In [16]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

In [18]:
# Tied/Shared Parameters

# We need to give the shared layer a name so that we can refer to its parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


=====================Parameter Initialization=======================

In [19]:
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1))
X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

In [20]:
# Built-in Initialization

def init_normal(module):
    if type(module) == nn.Linear:
        nn.init.normal_(module.weight, mean=0, std=0.01) #  initializes all weight parameters as Gaussian random variables with standard deviation 0.01
        nn.init.zeros_(module.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0131,  0.0030, -0.0118,  0.0016]), tensor(0.))

In [21]:
def init_constant(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 1) # given constant value
        nn.init.zeros_(module.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [22]:
def init_xavier(module):
    if type(module) == nn.Linear:
        nn.init.xavier_uniform_(module.weight)

def init_42(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.0509, -0.5759, -0.5115,  0.5238])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [23]:
# Custome initialization

def my_init(module):
    if type(module) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in module.named_parameters()][0])
        nn.init.uniform_(module.weight, -10, 10)
        module.weight.data *= module.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 8.5926,  0.0000, -8.8848,  7.0531],
        [-0.0000, -7.3913,  5.3600,  8.4965]], grad_fn=<SliceBackward0>)

In [31]:
print("Init", *("Hello", "World"))

Init Hello World


==================Lazy Initialization======================

In [37]:
import torch
from torch import nn
from d2l import torch as d2l

In [34]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

# At this point, the network cannot possibly know the dimensions of the input layer’s weights because the input dimension remains unknown.
# Consequently the framework has not yet initialized any parameters. We confirm by attempting to access the parameters below.

net[0].weight

<UninitializedParameter>

In [35]:
X = torch.rand(2, 20)
net(X)

net[0].weight.shape

torch.Size([256, 20])

In [ ]:
# The following method passes in dummy inputs through the network for a dry run to infer all parameter shapes and subsequently initializes the 
# parameters. It will be used later when default random initializations are not desired.


@d2l.add_to_class(d2l.Module)  #@save
def apply_init(self, inputs, init=None):
    self.forward(*inputs)
    if init is not None:
        self.net.apply(init)

=================Custom Layers===================

In [39]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [47]:
# Without Parameters


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()
    
layer = CenteredLayer()
layer(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [42]:
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean(), Y.shape

c:\Users\huiya\pytorch-basics\.venv311\Lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


(tensor(2.7940e-09, grad_fn=<MeanBackward0>), torch.Size([4, 128]))

In [44]:
# With Parameters

class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
    
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.5466,  1.1332,  1.1790],
        [-1.2105, -0.5504, -0.6399],
        [-1.1234,  0.0551,  0.0035],
        [ 0.2254, -1.6338,  0.2674],
        [-0.7772, -0.6681,  1.1908]], requires_grad=True)

In [45]:
linear(torch.rand(2, 5))

tensor([[0.0487, 0.6646, 0.5094],
        [0.0000, 0.0000, 0.0000]])

In [46]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.0000],
        [0.0039]])

================File I/O==============

In [48]:
import torch
from torch import nn
from torch.nn import functional as F

In [49]:
x = torch.arange(4)
torch.save(x, '..\\Data\\x-file')

In [50]:
x2 = torch.load('..\\Data\\x-file')
x2

tensor([0, 1, 2, 3])

In [55]:
y = torch.zeros(4)
torch.save([x, y],'..\\Data\\x-file')
x2, y2 = torch.load('..\\Data\\x-file')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [52]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [60]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.output = nn.LazyLinear(10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [64]:
torch.save(net.state_dict(), '..\\Data\\mlp.params')
# net.state_dict()

In [59]:
# To recover the model, we instantiate a clone of the original MLP model. Instead of randomly initializing the model parameters, we read the parameters stored in the file directly.

clone = MLP()
clone.load_state_dict(torch.load('..\\Data\\mlp.params'))
clone.eval()

MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

In [63]:
Y_clone = clone(X)
Y_clone == Y

tensor([[False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False]])

In [ ]:
X2 = torch.randn(size=(5, 20))
clone(X2) # works!

X3 = torch.randn(size=(5, 21))
# clone(X3) # Error:  mat1 and mat2 shapes cannot be multiplied (5x21 and 20x256)

=========================GPUs=========================

In [69]:
import torch
from torch import nn
from d2l import torch as d2l

In [ ]:
# In PyTorch, every array has a device; we often refer it as a context. So far, by default, all variables and associated computation have been assigned to the CPU. Typically, other contexts might be various GPUs.

